# SmartDesk AI - Full Notebook
Portfolio-ready single-file Jupyter Notebook with interactive backend, AI analysis, and frontend widgets.

In [ ]:
!pip install fastapi uvicorn sqlalchemy pydantic nest_asyncio requests openai ipywidgets matplotlib --quiet

In [ ]:
import nest_asyncio
nest_asyncio.apply()  # Allow FastAPI to run inside Jupyter

from fastapi import FastAPI
from pydantic import BaseModel
from sqlalchemy import Column, Integer, String, DateTime, create_engine, func
from sqlalchemy.orm import declarative_base, sessionmaker
import threading, uvicorn, requests, openai
import ipywidgets as widgets
from IPython.display import display, HTML
import matplotlib.pyplot as plt

In [ ]:
# Database setup (SQLite for notebook simplicity)
Base = declarative_base()
engine = create_engine("sqlite:///smartdesk_ai.db", connect_args={"check_same_thread": False})
SessionLocal = sessionmaker(bind=engine)
db = SessionLocal()

class Task(Base):
    __tablename__ = "tasks"
    id = Column(Integer, primary_key=True, index=True)
    title = Column(String)
    description = Column(String)
    status = Column(String, default="pending")
    created_at = Column(DateTime(timezone=True), server_default=func.now())

Base.metadata.create_all(bind=engine)

In [ ]:
# Pydantic schemas
class TaskCreate(BaseModel):
    title: str
    description: str

class TaskOut(TaskCreate):
    id: int
    status: str

In [ ]:
# FastAPI backend
app = FastAPI(title="SmartDesk AI Notebook Demo")

@app.post("/tasks/")
def create_task(task: TaskCreate):
    """Create a new task"""
    db_task = Task(**task.dict())
    db.add(db_task)
    db.commit()
    db.refresh(db_task)
    return TaskOut(id=db_task.id, title=db_task.title, description=db_task.description, status=db_task.status)

@app.get("/tasks/")
def list_tasks():
    """List all tasks"""
    tasks = db.query(Task).all()
    return [TaskOut(id=t.id, title=t.title, description=t.description, status=t.status) for t in tasks]

@app.post("/ai/analyze_task/")
def analyze_task(text: str):
    """Analyze task with AI (OpenAI GPT + Azure sentiment placeholder)"""
    openai.api_key = "<YOUR_OPENAI_KEY>"  # Replace with real key
    try:
        gpt_resp = openai.Completion.create(
            model="text-davinci-003",
            prompt=f"Analyze this task and suggest intent: {text}",
            max_tokens=50
        )
        intent = gpt_resp.choices[0].text.strip()
    except:
        intent = "DemoIntent"

    sentiment = "neutral"  # Placeholder for Azure Cognitive Services
    return {"intent": intent, "sentiment": sentiment, "summary": text}

In [ ]:
# Run FastAPI in background thread
def run_app():
    uvicorn.run(app, host="0.0.0.0", port=8000)

thread = threading.Thread(target=run_app, daemon=True)
thread.start()

In [ ]:
# Interactive frontend widgets
# Task creation widget
title_widget = widgets.Text(description="Title:")
desc_widget = widgets.Textarea(description="Description:")
create_btn = widgets.Button(description="Create Task")
output_create = widgets.Output()

def on_create_clicked(b):
    with output_create:
        output_create.clear_output()
        payload = {"title": title_widget.value, "description": desc_widget.value}
        response = requests.post("http://127.0.0.1:8000/tasks/", json=payload)
        print("Task Created:", response.json())
        title_widget.value = ""
        desc_widget.value = ""

create_btn.on_click(on_create_clicked)
display(HTML("<h3>Create New Task</h3>"))
display(title_widget, desc_widget, create_btn, output_create)

# Task list widget
list_btn = widgets.Button(description="List Tasks")
output_list = widgets.Output()

def on_list_clicked(b):
    with output_list:
        output_list.clear_output()
        response = requests.get("http://127.0.0.1:8000/tasks/")
        tasks = response.json()
        for t in tasks:
            print(t)

list_btn.on_click(on_list_clicked)
display(HTML("<h3>Task List</h3>"))
display(list_btn, output_list)

# AI analysis widget
ai_text = widgets.Textarea(description="Task Text:")
ai_btn = widgets.Button(description="Analyze Task")
output_ai = widgets.Output()

def on_ai_clicked(b):
    with output_ai:
        output_ai.clear_output()
        response = requests.post("http://127.0.0.1:8000/ai/analyze_task/", params={"text": ai_text.value})
        result = response.json()
        print("AI Analysis Result:", result)
        sentiment_map = {"positive": 1, "neutral": 0, "negative": -1}
        score = sentiment_map.get(result["sentiment"], 0)
        plt.bar(["Sentiment"], [score], color="skyblue")
        plt.ylim(-1, 1)
        plt.show()

ai_btn.on_click(on_ai_clicked)
display(HTML("<h3>AI Task Analyzer</h3>"))
display(ai_text, ai_btn, output_ai)